## Prepare data reader for a given chain and date

In [1]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros


# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=["daily_address_summary"],
    range_spec="@20241030:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-12-13 00:03:16 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=25 process=6571
2024-12-13 00:03:16 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=6571
2024-12-13 00:03:16 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76 process=6571
2024-12-13 00:03:16 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=37 process=6571
2024-12-13 00:03:16 [info     ] prepared 1 input batches.      filename=reader_bydate.py lineno=97 process=6571


## Run the model

This automatically registers the model outputs as duckdb tables.

In [ ]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory

execute_model_in_memory(
    duckdb_client=duckdb_client,
    model="daily_address_summary",
    data_reader=batch,
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
#
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

## Verify model results

In [ ]:
duckdb_client.sql("SELECT * FROM summary_v1 LIMIT 10")

### You can also convert the results to dataframes to inspect them in more familiar ways

In [ ]:
duckdb_client.sql("SELECT * FROM summary_v1 LIMIT 10").pl().head()